# BD-NSCA Multi-Turn Training on Kaggle

This notebook trains the Phi-3-mini model with QLoRA on Kaggle's free GPU (P100/T4).

## Setup Instructions:
1. Upload `train_multiturn.jsonl` to Kaggle as a dataset
2. Enable GPU: Settings → Accelerator → GPU P100 or T4 x2
3. Enable Internet: Settings → Internet → On
4. Run all cells

In [ ]:
# Install dependencies
!pip install -q transformers accelerate peft trl bitsandbytes datasets

In [ ]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

In [ ]:
# Configuration - adjust these as needed
CONFIG = {
    # Data path - UPDATE THIS to match your Kaggle dataset path
    "data_path": "/kaggle/input/npc-ai-training-data/train_multiturn.jsonl",
    
    # Model
    "base_model": "microsoft/Phi-3-mini-4k-instruct",
    
    # Training
    "num_epochs": 3,
    "batch_size": 4,  # Can use larger batch on P100 (16GB VRAM)
    "grad_accum": 4,
    "learning_rate": 2e-4,
    "max_seq_length": 2048,
    
    # LoRA
    "lora_r": 16,
    "lora_alpha": 32,
    "lora_dropout": 0.05,
    
    # Output
    "output_dir": "/kaggle/working/adapter_multiturn",
}

In [ ]:
import json
import warnings
warnings.filterwarnings("ignore")

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import LoraConfig, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
from datasets import Dataset

# Load dataset
def load_dataset(path):
    samples = []
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            if line.strip():
                samples.append(json.loads(line))
    print(f"Loaded {len(samples)} samples")
    return samples

raw_samples = load_dataset(CONFIG["data_path"])
formatted = [{"text": s.get("text", s.get("prompt", "") + s.get("completion", ""))} for s in raw_samples]
dataset = Dataset.from_list(formatted)
print(dataset)

In [ ]:
# Load model with 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    CONFIG["base_model"],
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager",
)
model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(CONFIG["base_model"], trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

model = prepare_model_for_kbit_training(model)
print("Model loaded!")

In [ ]:
# Configure LoRA
peft_config = LoraConfig(
    lora_alpha=CONFIG["lora_alpha"],
    lora_dropout=CONFIG["lora_dropout"],
    r=CONFIG["lora_r"],
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
)

# Configure training
sft_config = SFTConfig(
    output_dir=CONFIG["output_dir"],
    num_train_epochs=CONFIG["num_epochs"],
    per_device_train_batch_size=CONFIG["batch_size"],
    gradient_accumulation_steps=CONFIG["grad_accum"],
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=CONFIG["learning_rate"],
    fp16=True,  # Use fp16 on Kaggle GPUs
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
    max_length=CONFIG["max_seq_length"],
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=sft_config,
    formatting_func=lambda x: x["text"],
)

print("Trainer configured!")

In [ ]:
# Train!
print("Starting training...")
trainer.train()
print("Training complete!")

In [ ]:
# Save the adapter
trainer.model.save_pretrained(CONFIG["output_dir"])
tokenizer.save_pretrained(CONFIG["output_dir"])
print(f"Adapter saved to {CONFIG['output_dir']}")

# List saved files
import os
for f in os.listdir(CONFIG["output_dir"]):
    size = os.path.getsize(os.path.join(CONFIG["output_dir"], f)) / 1e6
    print(f"  {f}: {size:.2f} MB")

In [ ]:
# Download the adapter (creates a zip file)
import shutil
shutil.make_archive("/kaggle/working/adapter_multiturn", "zip", CONFIG["output_dir"])
print("Download adapter_multiturn.zip from the Output tab!")

## Quick Test (Optional)

In [ ]:
# Test the trained model
from peft import PeftModel

test_prompt = """System: You are friendly NPC villager. You share rumors and help adventurers.
Question: Have you heard any rumors lately?
Answer:"""

inputs = tokenizer(test_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))